In [1]:
import pandas as pd
ds = pd.read_csv('Panel.csv') 

def extract_details(row):
    try:
        # Parse JSON
        data = json.loads(row['response'])
        
        # Extract line items
        line_items = data.get('data', {}).get('line_items', [])
        
        extracted_data = []
        for item in line_items:
            # Extract relevant fields
            extracted_data.append({
                'claim_data_id': row['claim_data_id'],
                'intimation_number': row['intimation_number'],
                'bill_item_name': item.get('data', {}).get('description', {}).get('value', ''),
                'vitraya_master_category': item.get('data', {}).get('vitraya_master_category', {}).get('value', ''),
                'tariff_item_name': item.get('data', {}).get('tariff', {}).get('tariff_line_item', ''),
                'hospital_charged_amount': item.get('data', {}).get('amount', {}).get('value', 0.0),
                'quantity': item.get('data', {}).get('quantity', {}).get('value', 0.0),
                'irdai_payable': item.get('data', {}).get('tariff', {}).get('irdai_payable', False),
                'admissible_amount': item.get('data', {}).get('tariff', {}).get('admissible_amount', 0.0)
            })
        
        return extracted_data
    except json.JSONDecodeError:
        return []


ds_extracted = ds.apply(extract_details, axis=1)
ds_expanded = pd.DataFrame([item for sublist in ds_extracted for item in sublist])



In [3]:
print(ds_expanded)

      claim_data_id  intimation_number  \
0             12736      1112485205093   
1             12736      1112485205093   
2             12736      1112485205093   
3             12736      1112485205093   
4             12736      1112485205093   
...             ...                ...   
1489          10996      1112485189493   
1490          10996      1112485189493   
1491          10996      1112485189493   
1492          10996      1112485189493   
1493          12767      1112485206140   

                                         bill_item_name  \
0                                        Admission Fees   
1                                                 I C U   
2                                       Deluxe A/C Room   
3                                 Special with A/C Room   
4                                  Special Room Non A/C   
...                                                 ...   
1489  PHARMACY BILL (Department) : Bill :11582425/17173   
1490  PHARMACY BILL (De

,claim_data_id,intimation_number,bill_item_name,vitraya_master_category,tariff_item_name,hospital_charged_amount,quantity,irdai_payable,admissible_amount
1,12736,1112485205093,I C U,Medicines & Consumables,u dic,0.00,1.0,True,0.00
16,12736,1112485205093,Catheter,Medicines & Consumables,Catheter,0.00,1.0,False,0.00
17,12736,1112485205093,Nebulizar,Medicines & Consumables,Nebulizar,0.00,1.0,True,0.00
18,12736,1112485205093,Oxygen,Medicines & Consumables,Oxygen,0.00,1.0,True,0.00
19,12736,1112485205093,BSL,Medicines & Consumables,BSL,0.00,1.0,True,0.00
...,...,...,...,...,...,...,...,...,...
1458,10996,1112485189493,TOURNIQUET PER CASE,Medicines & Consumables,TOURNIQUET PER CASE,600.00,1.5,True,600.00
1459,10996,1112485189493,WARMER (OT),Medicines & Consumables,WARMER (OT),200.00,1.0,True,200.00
1461,10996,1112485189493,DRESSING MEDIUM,Medicines & Consumables,DRESSING MEDIUM,260.00,1.0,True,260.00
1462,10996,1112485189493,I.V.INFUSION,Medicines & Consumables,i.v.injection,210.00,3.0,False,0.00


In [8]:
df = ds_expanded.copy()

In [20]:
# Step 1: Filter the data where 'vitraya_master_category' is 'Medicines & Consumables'
filtered_df = df[df['vitraya_master_category'] == 'Medicines & Consumables']

# Step 2: Group by 'claim_data_id' and sum the 'hospital_charged_amount'
total_charged_amount = filtered_df.groupby('claim_data_id')['hospital_charged_amount'].sum().reset_index()
total_charged_amount.columns = ['claim_data_id', 'hospital charged medicine item total amount']

# Step 3: Merge the total charges back to the original dataframe
df = pd.merge(df, total_charged_amount, on='claim_data_id', how='left')

# If 'total_hospital_charged_amount' is NaN for rows where 'vitraya_master_category' is not 'Medicines & Consumables', set it to 0
df['hospital charged medicine item total amount'].fillna(0, inplace=True)

In [21]:
df[(df['vitraya_master_category'] == 'Medicines & Consumables') & (df['claim_data_id'] == 10996)]

,claim_data_id,intimation_number,bill_item_name,vitraya_master_category,tariff_item_name,hospital_charged_amount,quantity,irdai_payable,admissible_amount,total_hospital_charged_amount_x,total_hospital_charged_amount_y,hospital charged medicine item total amount
1432,10996,1112485189493,ETO-SMALL POUCH,Medicines & Consumables,eto-small pouch,60.00,2.0,False,0.00,15780.45,15780.45,15780.45
1433,10996,1112485189493,PLASMA STERILIZATION CHARGES - SMALL TYVEC POUCH,Medicines & Consumables,PLASMA STERILIZATION CHARGES - SMALL TYVEC POUCH,1300.00,4.0,False,0.00,15780.45,15780.45,15780.45
1434,10996,1112485189493,STEAM STERILIZATION CHARGES LARGE SET,Medicines & Consumables,STEAM STERILIZATION CHARGES LARGE SET,600.00,4.0,False,0.00,15780.45,15780.45,15780.45
1435,10996,1112485189493,STEAM STERILIZATION CHARGES MEDIUM SET,Medicines & Consumables,STEAM STERILIZATION CHARGES MEDIUM SET,500.00,5.0,False,0.00,15780.45,15780.45,15780.45
1454,10996,1112485189493,"O.T GASES, O2",Medicines & Consumables,"o.t gases, o2",510.00,3.0,False,0.00,15780.45,15780.45,15780.45
1455,10996,1112485189493,SPACE SUIT (PERSONAL PROTECTION SUIT),Medicines & Consumables,SPACE SUIT (PERSONAL PROTECTION SUIT),7200.00,1.5,True,7200.00,15780.45,15780.45,15780.45
1458,10996,1112485189493,TOURNIQUET PER CASE,Medicines & Consumables,TOURNIQUET PER CASE,600.00,1.5,True,600.00,15780.45,15780.45,15780.45
1459,10996,1112485189493,WARMER (OT),Medicines & Consumables,WARMER (OT),200.00,1.0,True,200.00,15780.45,15780.45,15780.45
1461,10996,1112485189493,DRESSING MEDIUM,Medicines & Consumables,DRESSING MEDIUM,260.00,1.0,True,260.00,15780.45,15780.45,15780.45
1462,10996,1112485189493,I.V.INFUSION,Medicines & Consumables,i.v.injection,210.00,3.0,False,0.00,15780.45,15780.45,15780.45


In [27]:
filtered_df = df[df['vitraya_master_category'] == 'Medicines & Consumables']

# Step 2: Group by 'claim_data_id' and sum the 'admissible_amount'
admissible_amount_total = filtered_df.groupby('claim_data_id')['admissible_amount'].sum().reset_index()
admissible_amount_total.columns = ['claim_data_id', 'admissible_amount_total_of_medicine_items']

# Step 3: Merge the total admissible amount back to the original dataframe
df = pd.merge(df, admissible_amount_total, on='claim_data_id', how='left')

# If 'admissible_amount_total_of_medicine_items' is NaN for rows where 'vitraya_master_category' is not 'Medicines & Consumables', set it to 0
df['admissible_amount_total_of_medicine_items'].fillna(0, inplace=True)

MergeError: Passing 'suffixes' which cause duplicate columns {'admissible_amount_total_of_medicine_items_x'} is not allowed.

In [28]:
# Step 1: Group by 'claim_data_id' and calculate totals
totals = df.groupby('claim_data_id').agg(
    total_hospital_charged_amount=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum'),
    total_admissible_amount=pd.NamedAgg(column='admissible_amount', aggfunc='sum')
).reset_index()

# Step 2: Merge the totals back to the original DataFrame
df = pd.merge(df, totals, on='claim_data_id', how='left')

print(df)

      claim_data_id  intimation_number  \
0             12736      1112485205093   
1             12736      1112485205093   
2             12736      1112485205093   
3             12736      1112485205093   
4             12736      1112485205093   
...             ...                ...   
1489          10996      1112485189493   
1490          10996      1112485189493   
1491          10996      1112485189493   
1492          10996      1112485189493   
1493          12767      1112485206140   

                                         bill_item_name  \
0                                        Admission Fees   
1                                                 I C U   
2                                       Deluxe A/C Room   
3                                 Special with A/C Room   
4                                  Special Room Non A/C   
...                                                 ...   
1489  PHARMACY BILL (Department) : Bill :11582425/17173   
1490  PHARMACY BILL (De

In [32]:
 #Step 1: Calculate total hospital charged amount and total admissible amount for 'Medicines & Consumables'
med_consumables = df[df['vitraya_master_category'] == 'Medicines & Consumables']
med_consumables_totals = med_consumables.groupby('claim_data_id').agg(
    admissible_amount_total_of_medicine_items=pd.NamedAgg(column='admissible_amount', aggfunc='sum'),
    total_hospital_charged_amount_meds=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum')
).reset_index()

# Step 2: Calculate total hospital charged amount and total admissible amount for all items
all_items_totals = df.groupby('claim_data_id').agg(
    total_hospital_charged_amount_all=pd.NamedAgg(column='hospital_charged_amount', aggfunc='sum'),
    total_admissible_amount_all=pd.NamedAgg(column='admissible_amount', aggfunc='sum')
).reset_index()

# Step 3: Merge all totals into a single DataFrame
final_totals = pd.merge(med_consumables_totals, all_items_totals, on='claim_data_id', how='outer')

# Fill NaNs with 0 if necessary
final_totals.fillna(0, inplace=True)

    claim_data_id  admissible_amount_total_of_medicine_items  \
0           10996                                   12600.45   
1           11267                                  419387.99   
2           12095                                   10752.00   
3           12154                                   21581.63   
4           12217                                   55000.00   
5           12407                                    8545.48   
6           12423                                    3704.00   
7           12429                                  240022.00   
8           12473                                   14459.69   
9           12499                                    8487.00   
10          12537                                   16860.92   
11          12587                                     549.80   
12          12605                                    3590.60   
13          12623                                   46644.11   
14          12634                       

In [33]:
final_totals

,claim_data_id,admissible_amount_total_of_medicine_items,total_hospital_charged_amount_meds,total_hospital_charged_amount_all,total_admissible_amount_all
0,10996,12600.45,15780.45,582288.91,578428.91
1,11267,419387.99,448080.95,986873.00,920645.04
2,12095,10752.00,10752.00,57848.00,54218.00
3,12154,21581.63,31888.69,164005.97,148278.91
4,12217,55000.00,55000.00,56000.00,55000.00
5,12407,8545.48,9195.48,80825.48,78895.48
6,12423,3704.00,3704.00,17349.00,15349.00
7,12429,240022.00,240022.00,290157.00,289857.00
8,12473,14459.69,24543.58,73717.02,53428.13
9,12499,8487.00,8487.00,21987.00,21487.00
